In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
skincare = pd.read_csv(f"{DATA_PATH}아모레크롤링_스킨케어.csv")

In [7]:
makeup = pd.read_csv(f"{DATA_PATH}아모레크롤링_메이크업_완료.csv")

In [15]:
food = pd.read_csv(f"{DATA_PATH}뷰티푸드 별점.csv")

In [18]:
perfume = pd.read_csv(f"{DATA_PATH}향수 별점.csv")

In [21]:
thing = pd.read_csv(f"{DATA_PATH}소품도구 별점.csv")

In [22]:
frames = [skincare, makeup, food, perfume, thing]
df = pd.concat(frames, ignore_index=True)
df

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,0103*******,5,2022.09.27,40대,여성,지성,모공,남편이 극지성이에요. 원래 수퍼화산송이 미셀라만 꾸준히 사용해 오다가 지성 전용이라...
1,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,lsm2***,5,2022.08.15,40대,여성,건성,건조함,여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 ...
2,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,gpdl***,5,2022.12.24,20대,여성,복합성,트러블,이니스프리 폼클렌징은 피부에 자극적이지 않아 좋아요 선크림만 사용해도 꼭 폼까지 써...
3,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,zoll***,5,2022.09.30,30대,여성,건성,건조함,저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁...
4,스킨케어,이니스프리,화산송이 모공 바하 클렌징 폼 150g,snow****,5,2022.09.30,50대 이상,여성,지성,모공,여름이라서 모공이 넓어지는것 같아서 구매해서 사용해보니 촉촉하고 피부땡기지 않아서 ...
...,...,...,...,...,...,...,...,...,...,...,...
203799,소품도구,피카소,103A 파우더,suy*,5,2019-08-07,20대,여성,NaN,NaN,명불허전이에요~ 파우더나 블러셔 용도로 매우 잘쓰고 있어요!
203800,소품도구,피카소,103A 파우더,yyou****,5,2019-06-25,20대,여성,NaN,NaN,부드럽고 파우더 바르기 좋아요! 다른 브러쉬로 바르면 파우더가 조금 떠서 피카소 브...
203801,소품도구,피카소,103A 파우더,amil****,5,2019-05-18,40대,여성,NaN,NaN,피카소브러쉬 너무좋아요 계속사게되요 모질도 너무 부드럽고 ~ 이번에 행사해서 할인...
203802,소품도구,피카소,103A 파우더,vbgu*****,4,2019-04-30,30대,여성,NaN,NaN,가격은 좀 비싸지만 생각보다 괜찮은것 같아요 배송은 하루만에 받았구요 보호케이스가 ...


In [23]:
def chatbot(df):
    model_name = "noahkim/KoT5_news_summarization"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        # 여러 키워드에 대한 처리
        keywords = title.split()
        condition = lambda x: all(keyword.lower() in x.lower() for keyword in keywords)
        # 중복 제거를 위해 unique() 사용
        matching_titles = [t for t in df['상품명'].unique().tolist() if condition(t)]
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        # 사용자가 상품을 선택하도록 함
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        # 선택한 상품의 리뷰 가져오기
        reviews = df[df['상품명'] == selected_title]['리뷰'].to_list()
        reviews_text = ' '.join(reviews)
        # 리뷰 요약
        inputs = tokenizer(reviews_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=40, max_length=200, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(f"{selected_title}의 전체 리뷰 요약: {summary}")

In [24]:
chatbot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 화산송이
일치하는 상품명 목록:
1. 화산송이 모공 바하 클렌징 폼 150g
2. 수퍼 화산송이 모공 마스크 100ml
3. 화산송이 모공 바하 클렌징폼  250g
상품을 선택하세요 (번호 입력): 1
화산송이 모공 바하 클렌징 폼 150g의 전체 리뷰 요약: 여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 세안 해주면 좋은점이 뾰루지가 안나고 뽀드득하게 씻겨줘서 좋고 피지과도할때 딥 클렌징폼이라 그런지 피부에 자극이 없는건 아니지만 격일로 쓰기에 좋은거같아요.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.


In [25]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 25.0 MB/s eta 0:00:00


In [26]:
from konlpy.tag import Okt
from collections import Counter

In [27]:
def keyword_bot(df):
    while True:
        title = input("상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
        if title.lower() in ['q', 'quit']:
            print("챗봇을 종료합니다.")
            break
        matching_titles = df[df['상품명'].str.contains(title, case=False)]['상품명'].unique().tolist()
        if not matching_titles:
            print("해당 상품이 없습니다.")
            continue
        print("일치하는 상품명 목록:")
        for idx, matching_title in enumerate(matching_titles, start=1):
            print(f"{idx}. {matching_title}")
        selected_idx = int(input("상품을 선택하세요 (번호 입력): "))
        selected_title = matching_titles[selected_idx - 1]
        keyword(selected_title)  # 상품명을 함수에 전달하여 실행

def keyword(title):
    texts = df[df['상품명'] == title]['리뷰'].to_list()
    okt = Okt()
    nouns_list = []
    for text in texts:
        nouns = okt.nouns(text)
        nouns_list.extend(nouns)

    count = Counter(nouns_list)

    # 명사 빈도 카운트
    # 상위 100개
    noun_list = count.most_common(100)
    for v in noun_list:
        if any(keyword in v[0] for keyword in ['지성','수부지', '건성', '복합성','여드름','잡티','피부톤','탄력','피지','모공','블랙헤드','민감성','봄웜','웜톤','봄웜톤','봄라이트','봄라','봄브라이트','여쿨','여름쿨톤','여라','여름라이트','여름뮤트','여뮽','뮤트','뮤트톤','여름브라이트','가을웜톤','가을뮤트','갈딥','가을딥','갈웜','겨쿨','겨울브라이트','겨울딥','쿨톤']):
            print("이 제품은 "+v[0]+" 타입에게 추천하는 제품입니다.")

In [28]:
keyword_bot(df)

상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): 화산송이
일치하는 상품명 목록:
1. 화산송이 모공 바하 클렌징 폼 150g
2. 수퍼 화산송이 모공 마스크 100ml
3. 화산송이 모공 바하 클렌징폼  250g
상품을 선택하세요 (번호 입력): 1
이 제품은 모공 타입에게 추천하는 제품입니다.
이 제품은 피지 타입에게 추천하는 제품입니다.
이 제품은 여드름 타입에게 추천하는 제품입니다.
이 제품은 건성 타입에게 추천하는 제품입니다.
상품명을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): q
챗봇을 종료합니다.
